In [1]:
import numpy as np
import pandas as pd
from psycopg2 import OperationalError
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, min, max
import gzip
import json
import pymysql
import psycopg2
from psycopg2 import extras

#  Using Jupyter notebook create a program to read the million reviews and get the following.

In [2]:

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}  # Corrected variable name from df1 to df
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

path = "../TensorIot/AMAZON_FASHION_5.json.gz"  # Replace with the actual path to your JSON.gz file
df= getDF(path)


#  Transform: change the date MM-DD-YYYY format.

In [3]:
# convert the date column to datetime format
df['reviewTime'] = pd.to_datetime(df['reviewTime'])

# change the datetime format
df['reviewTime'] = df['reviewTime'].dt.strftime('%m-%d-%Y')

In [4]:
# Show a desired data frame operation which you learnt recently.

In [5]:
df['reviewTime']

0       09-04-2015
1       09-04-2015
2       09-04-2015
3       09-04-2015
4       09-04-2015
           ...    
3171    07-02-2018
3172    06-28-2018
3173    06-25-2018
3174    06-20-2018
3175    06-16-2018
Name: reviewTime, Length: 3176, dtype: object

# Convert the whole file into Parquet file after transforming.

In [6]:
# Convert the whole file into Parquet file after transforming.

df.to_parquet("Task2.paraquet")

# Connect to Database

In [7]:
# pip install ipython-sql 

In [8]:
# pip install psycopg2

In [9]:
""" create tables in the PostgreSQL database"""

commands = (
       
        """
        CREATE TABLE amazon_fashion (
                overall float,
                verified boolean,
                reviewTime VARCHAR ,
                reviewerID VARCHAR,
                asin VARCHAR,
                style VARCHAR,
                reviewerName VARCHAR,
                reviewText VARCHAR,
                summary VARCHAR,
                unixReviewTime INTEGER,
                vote VARCHAR,
                image VARCHAR,
                review_length INTEGER                    
        );
        """)
   

In [10]:
# connection with postgresql, made to database commit , Insert data and close connection.

def execute_values(conn, df, table):
    # Convert dictionary columns to JSON strings
    
    for col in df.select_dtypes(include=['object']).columns:
        if isinstance(df[col].iloc[0], dict):
            df[col] = df[col].apply(json.dumps)

    
    tuples = [tuple(row) for row in df.to_numpy()]              # Convert DataFrame to a list of tuples

    
    cols = ','.join(list(df.columns))                           # Get column names

    
    query = f"INSERT INTO {table}({cols}) VALUES %s"            # SQL query to execute

    
    cursor = conn.cursor()                                      # Create a cursor

    try:
        
        psycopg2.extras.execute_values(cursor, query, tuples)   # Use execute_values to insert data
        
        conn.commit()                                           # Commit the transaction
        print("Data inserted successfully.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        
        conn.rollback()                                        # Rollback in case of an error
    finally:
        
        cursor.close()                                         # Close the cursor
        

# Replace these variables with your actual PostgreSQL connection details
db_config = {
    'host': 'localhost',
    'database': 'TensorIot',
    'user': 'postgres',
    'password': '010101',
}


table_name = 'amazon_fashion'



# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(**db_config)
    execute_values(conn, df, table_name)
except OperationalError as e:
    print(f"Error connecting to the database: {e}")
finally:
    # Close the connection outside the try-except block
    if conn is not None:
        conn.close()


Data inserted successfully.


In [11]:
db_config = {
    'host': 'localhost',
    'database': 'TensorIot',
    'user': 'postgres',
    'password': '010101',
}

query = f"SELECT * FROM amazon_fashion limit 5;"            # SQL query to execute
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()                                      # Create a cursor

try:

    cursor.execute(query)   # Use execute_values to insert data

    conn.commit()                                           # Commit the transaction
    print(cursor.fetchall())
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")

    conn.rollback()                                        # Rollback in case of an error
finally:

    cursor.close()                                         # Close the cursor


[(5.0, True, '09-04-2015', 'ALJ66O1Y6SLHA', 'B000K2PJ4K', '{"Size:": " Big Boys", "Color:": " Blue/Orange"}', 'Tonya B.', 'Great product and price!', 'Five Stars', 1441324800, 'NaN', 'NaN', 24), (5.0, True, '09-04-2015', 'ALJ66O1Y6SLHA', 'B000K2PJ4K', '{"Size:": " Big Boys", "Color:": " Black (37467610) / Red/White"}', 'Tonya B.', 'Great product and price!', 'Five Stars', 1441324800, 'NaN', 'NaN', 24), (5.0, True, '09-04-2015', 'ALJ66O1Y6SLHA', 'B000K2PJ4K', '{"Size:": " Big Boys", "Color:": " Blue/Gray Logo"}', 'Tonya B.', 'Great product and price!', 'Five Stars', 1441324800, 'NaN', 'NaN', 24), (5.0, True, '09-04-2015', 'ALJ66O1Y6SLHA', 'B000K2PJ4K', '{"Size:": " Big Boys", "Color:": " Blue (37867638-99) / Yellow"}', 'Tonya B.', 'Great product and price!', 'Five Stars', 1441324800, 'NaN', 'NaN', 24), (5.0, True, '09-04-2015', 'ALJ66O1Y6SLHA', 'B000K2PJ4K', '{"Size:": " Big Boys", "Color:": " Blue/Pink"}', 'Tonya B.', 'Great product and price!', 'Five Stars', 1441324800, 'NaN', 'NaN', 